In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats, signal, optimize

#potentiall remove at some point
import pickle
import pandas as pd
from tqdm import tqdm

# Global parameters

In [ ]:
# global parameters
dt=1

# Implementation of measures

### 1. Find smallest $h$ for which the overlap with the zero-input case is less than discrimination error $\varepsilon$
Note: $a(0)=0$ for all $\lambda$ and $\mu$ such that $P(o)=\mathcal{N}(0,\sigma)$

In [ ]:
def calc_overlap(pmf1, pmf2):
    """
    calculates the overlap between two discrete probability mass functions
    ATTENTION: needs user to ensure that domains are identical!
    """
    assert len(pmf1) == len(pmf2)
    return np.sum(np.minimum(pmf1, pmf2)) * 0.5


def find_discriminable_inputs(pmf, h_range, epsilon: float, start="left"):
    """
    Determine all inputs h in range h_range such that the overlap between all pmfs is less than epsilon
    The pmfs of h_range[0] and h_range[1] sever as boundaries

    # Parameter
    - pmf: function
    - h_range: array-like with length two
    - epsilon: float
        discrimination error that specifies maximal overlap between two probability mass functions
    """
    assert len(h_range) == 2
    h_left, h_right = h_range
    hs = []
    if start == "left":
        h_ref = h_left
        pmf_end = pmf(h_right)
    elif start == "right":
        h_ref = h_right
        pmf_end = pmf(h_left)
    pmf_ref = pmf(h_ref)

    while True:
        def func(h):
            return calc_overlap(pmf_ref, pmf(h)) - epsilon

        try:
            if start == "left":
                h_new = optimize.bisect(func, h_ref, h_right)
            elif start == "right":
                h_new = optimize.bisect(func, h_left, h_ref)
            pmf_new = pmf(h_new)
            if calc_overlap(pmf_end, pmf_new) < epsilon:
                hs.append(h_new)
                h_ref = h_new
                pmf_ref = pmf_new
            else:
                break
        except:
            break
    return hs


def dynamic_range(h_range):
    """
    Calculate the dynamic range from the range h_range
    """
    assert len(h_range) == 2
    h_left, h_right = h_range
    return 10 * (np.log10(h_right) - np.log10(h_left))

In [ ]:
# test discriminable intervals
N = int(1e4)
mu = 0.2
lam = 0.99999
sigma=1e-2
Xs = np.arange(-4*sigma*N,N+4*sigma*N)
epsilon=0.02

def pmf(h):
    # see below for formal derivation, here just as a test
    A = mu*(1-np.exp(-h*dt))/(1-lam*(1-mu)-lam*mu*np.exp(-h*dt)) * N
    # in domain with delta = 1
    delta = 1
    return stats.norm.pdf(Xs, A, sigma*N)*delta

h_range=[0,1e3]
pmf_ref_left = pmf(h_range[0])
pmf_ref_right = pmf(h_range[1])

plt.plot(Xs,pmf_ref_left, color="gray")
plt.plot(Xs,pmf_ref_right, color="gray")

hs_left = find_discriminable_inputs(pmf,h_range, epsilon, start="left")
print(hs_left)
for h in hs_left:
    plt.plot(Xs,pmf(h), color="blue")

hs_right = find_discriminable_inputs(pmf,h_range, epsilon, start="right")
print(hs_right)
for h in hs_right:
    plt.plot(Xs,pmf(h), color="orange")
#plt.legend()

# Consider network output with Gaussian noise
The output of the network is the sum of the network activity and Gaussian noise $\eta\sim\mathcal{N}(0,1)$ such that 
$$ o = a + \sigma\eta $$
The output is thus a convolution between $P(a)$ and $\mathcal{N}(0,\sigma)$.

For $T\to\infty$ the distribution of activity is a delta-distribution such that the the output distribution becomes a shifted Gaussian
$$ P(o) = \mathcal{N}(a(h|\lambda,\mu), \sigma)$$

# Analytic solution for $T\to\infty$
For the $T\to\infty$ case, we have a one-to-one mapping between input $h$ and network activity $a$, given by 
$$a(h|\lambda, \mu) = \frac{\mu\left(1-e^{-h\Delta t}\right)}{1-\lambda(1-\mu)-\lambda\mu e^{-h\Delta t}}$$

In [ ]:
def a(h,lam,mu):
    return mu*(1-np.exp(-h*dt))/(1-lam*(1-mu)-lam*mu*np.exp(-h*dt))

hs=np.logspace(-5,1,20)
colors=['red','blue','orange','green']
for i,lam in enumerate([0,0.9,0.99,0.999]):
    plt.plot(hs*0.2, a(hs, lam, 0.2), label='{}'.format(lam), color=colors[i], alpha=0.5)
    plt.plot(hs, a(hs, lam, 1.0), color=colors[i])
plt.xscale("log")
plt.xlabel("input")
plt.ylabel("activity")
plt.legend()

In [ ]:
# network parameters
N = int(1e4)
mu = 0.2
sigma = 1e-2
Xs = np.arange(-4 * sigma * N, N + 4 * sigma * N)
epsilon = 0.2
lams = 1 - np.logspace(-4, 0, 30)

# analysis parameters
drs = np.zeros(len(lams))
nds = np.zeros(len(lams))
mis = np.zeros(len(lams))
h_range = [0, 1e3]
# attempt to match previous data analysis
logh = np.arange(-7, 1.5, 0.05)  # 170 elements
for i, lam in tqdm(enumerate(lams), total=len(lams)):
    # distribution of noisy output
    def pmf_o_given_h(h):
        A = a(h, lam, mu) * N
        return stats.norm.pdf(Xs, A, sigma * N)

    hs_left = find_discriminable_inputs(pmf_o_given_h, h_range, epsilon)
    hs_right = find_discriminable_inputs(pmf_o_given_h, h_range, epsilon, start="right")
    drs[i] = dynamic_range((hs_left[0], hs_right[0]))
    nds[i] = 0.5 * (len(hs_left) + len(hs_right))

# plot the number of discriminable inputs for different values of epsilon and T in a main plot with an inset
markers = ["o", "s", "^", "P", "d"]
legends = ["$1$ ms", "$10$ ms", "$10^2$ ms", "$10^3$ ms", "$10^4$ ms"]

fig, ax = plt.subplots(1, 2, figsize=(10, 4))
ax[0].plot(1 - lams, nds)
ax[0].set_xlabel("branching parameter $\lambda$")
ax[0].set_ylim(0, 65)
ax[0].set_ylabel("number of discriminable inputs $n_d$")
ax[0].set_xscale("log")
ax[0].set_xticks([1e-4, 1e-3, 1e-2, 1e-1, 1e0])
ax[0].set_xticklabels(["0.9999", "0.999", "0.99", "0.9", "0"])
ax[0].invert_xaxis()
ax[1].plot(1 - lams, drs)
ax[1].set_xlabel("branching parameter $\lambda$")
ax[1].set_ylim(0, 40)
ax[1].set_ylabel("dynamic range $\Delta$")
ax[1].set_xscale("log")
ax[1].set_xticks([1e-4, 1e-3, 1e-2, 1e-1, 1e0])
ax[1].set_xticklabels(["0.9999", "0.999", "0.99", "0.9", "0"])
ax[1].invert_xaxis()

# Analysis of data

In [ ]:
# load parameters fo beta distribution from file
def load_beta_params(filename):
    with open(filename, 'rb') as f:
        beta_params = pickle.load(f)
    return beta_params

beta_params = load_beta_params('../data/betaParams/subpop_k=100/epsilon=1.00e-01_subFix_window=1_realization=0.pkl')
print(beta_params.keys())

# plot beta parameters for different values of epsilon in two parallel subplots (a and b)
fig, axes = plt.subplots(2, 1, figsize=(10, 5))
epsilons = [1e-1, 1e-2, 1e-3, 1e-4]
epsilons = np.logspace(-4, 0, 9).tolist()
for epsilon in epsilons:
    beta_params = load_beta_params('../data/betaParams/subpop_k=100/epsilon={:.2e}_subFix_window=1_realization=0.pkl'.format(epsilon))
    # filter out invalid values (a,b < 0)
    mask = (beta_params['a'] > 0) & (beta_params['b'] > 0)
    loghs = beta_params['logh'][mask]
    valas = beta_params['a'][mask]
    valbs = beta_params['b'][mask]
    #plot
    axes[0].scatter(loghs, valas, label='epsilon={}'.format(epsilon))    
    axes[1].scatter(loghs, valbs, label='epsilon={}'.format(epsilon))    

axes[0].set_xlabel("log h")
axes[0].set_ylabel("a")
axes[1].set_xlabel("log h")
axes[1].set_ylabel("b")
axes[0].set_yscale("log")
axes[1].set_yscale("log")
#axes[1].legend()
plt.tight_layout()

Fit the available data with a deep neural network that maps system parameters (h,epsilon) to beta-distribution parameters (a,b)

In [ ]:
import torch
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print (x)
else:
    print ("MPS device not found.")

In [ ]:
import sklearn
from sklearn.preprocessing import MinMaxScaler
# load data
x_data = []
y_data = []

def get_data(realization, epsilons=np.logspace(-4, 0, 9).tolist(), shuffle=1234):
    X1, X2 = np.empty(shape=(0,)), np.empty(shape=(0,))
    Y1, Y2 = np.empty(shape=(0,)), np.empty(shape=(0,))
    for epsilon in epsilons:
        beta_params = load_beta_params('../data/betaParams/subpop_k=100/epsilon={:.2e}_subFix_window=1_realization={:d}.pkl'.format(epsilon, realization))
        # filter out invalid values where either a or b are negative
        mask = (beta_params['a'] > 0) & (beta_params['b'] > 0)
        loghs = beta_params['logh'][mask]
        loges = np.log(epsilon) * np.ones_like(loghs)
        logas = np.log(beta_params['a'][mask])
        logbs = np.log(beta_params['b'][mask])
        X1 = np.append(X1, loghs)
        X2 = np.append(X2, loges)
        Y1 = np.append(Y1, logas)
        Y2 = np.append(Y2, logbs)
    # shuffle with random mask
    mask = np.arange(len(X1))
    # if shuffle is not False:
    if shuffle is not None:
        np.random.seed(shuffle)
        np.random.shuffle(mask)
    x_data = np.stack((X1[mask], X2[mask]), axis=1)
    y_data = np.stack((Y1[mask], Y2[mask]), axis=1)
    print(realization, x_data.shape, y_data.shape)
    return x_data, y_data, 
get_data(1, shuffle=None)

We follow here the [simple example](https://notebook.community/kit-cel/lecture-examples/mloc/ch3_Deep_Learning/pytorch/function_approximation_with_MLP) using pytorch

Define the model

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden1 = nn.Linear(2, 42)
        self.act1 = nn.Tanh()
        self.hidden2 = nn.Linear(42, 42)
        self.act2 = nn.Tanh()
        self.hidden3 = nn.Linear(42, 42)
        self.act3 = nn.Tanh()
        self.output = nn.Linear(42, 2)
        #self.act_output = nn.Sigmoid()
 
    def forward(self, x):
        x = self.act1(self.hidden1(x))
        x = self.act2(self.hidden2(x))
        x = self.act3(self.hidden3(x))
        x = self.output(x)
        return x
 

def train_model_simple(X_train, Y_train, epochs):
    """
    Trains a simple neural network with one hidden layer with units neurons
    """
    # prepare data
    X_train_tensor = torch.from_numpy(X_train).float()
    Y_train_tensor = torch.from_numpy(Y_train).float()

    # create model
    model = NeuralNetwork()

    # Adam and MSE Loss
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    loss_fn = nn.MSELoss(reduction='mean')
    
    # main loop    
    history_loss = []
    for epoch in tqdm(range(epochs)):
        yhat = model(X_train_tensor)
        loss = loss_fn(yhat, Y_train_tensor)
        history_loss.append(loss.item())
        # compute gradients
        loss.backward() 
        # carry out one optimization step with Adam
        optimizer.step()   
        # reset gradients to zero
        optimizer.zero_grad()

    return model, history_loss

# rescale data (can use the same scaler multiple times using fit_transform function)
X_scaler = MinMaxScaler(feature_range=(-1, 1))
Y_scaler = MinMaxScaler(feature_range=(-1, 1))
X, Y = get_data(0)
X_scaled = X_scaler.fit_transform(X)
Y_scaled = Y_scaler.fit_transform(Y)

# train model
model, loss = train_model_simple(X_scaled, Y_scaled, 10000)
plt.plot(loss)
plt.xlabel('epoch')
plt.ylabel('loss')
plt.yscale('log')
plt.show()


In [ ]:
import itertools
import matplotlib.pyplot as plt
#compare to data
X, Y = get_data(0, shuffle=None)
# mesh grid for contour plot
loghs = np.linspace(-7, 1.5, 170)
loges = np.linspace(-4, 0, 9)
X_mesh = np.meshgrid(loghs, loges)
#print(X_mesh.shape)

# compare to data
fig, axes = plt.subplots(2, 1, figsize=(10, 5))
X,Y = get_data(0, shuffle=None)
# plot for all epsilon (always same color!)

for epsilon in np.logspace(-4, 0, 9).tolist()[:-1]:
    # plot data
    mask = X[:,1]==np.log(epsilon)
    ref = axes[0].scatter(X[mask,0], np.exp(Y[mask, 0]), label='epsilon={}'.format(epsilon))
    color = ref.get_facecolor()[0]
    axes[1].scatter(X[mask,0], np.exp(Y[mask, 1]), label='epsilon={}'.format(epsilon), color=color)
    # plot model
    loghs = np.linspace(-7, 1.5, 200)
    loges = np.log(epsilon) * np.ones_like(loghs)
    X_model = np.stack((loghs, loges), axis=1)
    X_model_scaled = X_scaler.transform(X_model)
    Y_model_scaled = model(torch.from_numpy(X_model_scaled).float()).detach().numpy()
    Y_model = Y_scaler.inverse_transform(Y_model_scaled)
    axes[0].plot(loghs, np.exp(Y_model[:, 0]), color=color)
    axes[1].plot(loghs, np.exp(Y_model[:, 1]), color=color)

axes[0].set_xlabel("log h")
axes[0].set_ylabel("a")
axes[1].set_xlabel("log h")
axes[1].set_ylabel("b")
axes[0].set_yscale("log")
axes[1].set_yscale("log")
#axes[1].legend()
plt.tight_layout()


From the interpolation model we can obtain estimates of the dynamic range and the resolution

In [ ]:
a = 1e-1
b = 100
delta = 1/N
support = np.arange(-1-4*sigma, 1 + 4*sigma, delta)
pmf_beta = stats.beta.pdf(support, a, b)*delta
pmf_norm = stats.norm.pdf(support, 0, sigma)*delta
print(np.sum(pmf_beta), np.sum(pmf_norm))
print(pmf_beta)
# convolution with a Gaussian distribution at every point of the domain
conv = signal.fftconvolve(pmf_beta, pmf_norm, mode='same')
print(np.sum(conv))
mask = conv > 1e-5
plt.plot(support[mask], pmf_beta[mask])
plt.plot(support[mask], pmf_norm[mask])
plt.plot(support[mask], conv[mask])


In [ ]:
# test discriminable intervals (TODO: here is sth still not quite as it is supposed to be ... overlap between distributions is not epsilon) ... am I missing the convolution!!
N = int(1e4)
mu = 0.2
lam = 0.999
sigma=1e-2
# need delta 1/N to be consistent with the overlap error that is defined on the level of the pmf with activity unit of neurons?
delta = 1/N
domain = np.arange(-4*sigma - 1, 1 + 4*sigma, delta)
epsilon=0.02

def pmf(h, convolve=True):
    # distribution is given by Beta-distribution specified by a and b
    X = np.log(np.array([h,epsilon]))
    X_scaled = X_scaler.transform(X.reshape(-1,1).T)
    Y_scaled = model(torch.from_numpy(X_scaled).float()).detach().numpy()
    loga,logb = Y_scaler.inverse_transform(Y_scaled).T.reshape(-1)
    a,b = np.exp(loga), np.exp(logb)
    if convolve:
        pmf_beta = stats.beta.pdf(domain, a, b)*delta
        pmf_norm = stats.norm.pdf(domain, 0, sigma)*delta
        # convolution with a Gaussian distribution at every point of the domain
        return np.convolve(pmf_beta, pmf_norm, mode="same")
    else:
        return stats.beta.pdf(domain, a, b)*delta

h_range=[1e-7,1e3]
pdf_ref_left = stats.norm.pdf(domain, 0, sigma)*delta
pdf_ref_right = stats.norm.pdf(domain, 1, sigma)*delta

hs_left = find_discriminable_inputs(pmf, h_range, epsilon, start="left")
print("hs_left: ", hs_left)
for h in hs_left:
    plt.plot(domain,pmf(h), color="blue")

hs_right = find_discriminable_inputs(pmf, h_range, epsilon, start="right")
print("hs_right: ", hs_right)
for h in hs_right:
    plt.plot(domain,pmf(h), color="orange")

plt.plot(domain,pdf_ref_left, color="gray")
plt.plot(domain,pdf_ref_right, color="gray")
plt.xlim(-4*sigma,1)

# plt.legend()

In [ ]:
epsilons = np.logspace(-4, 0, 30)

# analysis parameters
drs = np.zeros(len(epsilons))
nds = np.zeros(len(epsilons))
mis = np.zeros(len(epsilons))
# need to exclude the zero here because of logh fit
h_range = [1e-7, 1e1]
for i, epsilon in enumerate(epsilons):
    print(i,epsilon)
    # distribution is given by Beta-distribution specified by a and b
    def pmf_o_given_h(h):
        X = np.log(np.array([h,epsilon]))
        X_scaled = X_scaler.transform(X.reshape(-1,1).T)
        Y_scaled = model(torch.from_numpy(X_scaled).float()).detach().numpy()
        loga,logb = Y_scaler.inverse_transform(Y_scaled).T.reshape(-1)
        a,b = np.exp(loga), np.exp(logb)
        return stats.beta.pdf(Xs/N, a, b)
        
    hs_left = find_discriminable_inputs(pmf_o_given_h, h_range, epsilon)
    hs_right = find_discriminable_inputs(pmf_o_given_h, h_range, epsilon, start="right")
    print(hs_left,hs_right)
    drs[i] = dynamic_range((hs_left[0], hs_right[0]))
    nds[i] = 0.5 * (len(hs_left) + len(hs_right))



## TODO:
- Develop an approximation to the finite-time solution as a correction to the infinite T limit?
- Fit this?
- Or take absolute maximum? -> as inset?
- Only Fig n_d and DR ... Lets try to replot with development of maxima as inset
